## DATA ENGINEER INTERN ASSIGNMENT 

In [23]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup

In [27]:
url = 'https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population'
page = requests.get(url)
data = page.text
soup = BeautifulSoup(data, 'lxml')
state = soup.find('div',class_ ='wikitable sortable jquery-tablesorter')
table = soup.findChildren('table')[4]
columns = table.find_all('tr')

all_cols = []
all_links = []

for col in columns:
    cols=col.find_all('td')
    link_can = col.find('a',href = True)
    all_links.append('https://en.wikipedia.org'  + link_can['href'])    
    cols=[x.text.strip() for x in cols]
    all_cols.append(cols)
final_table = pd.DataFrame(all_cols, columns=['Rank', 'city','state','2017_estimate','2018_census','change','2016_land_area_miles','2016_land_area_km','2016_population_density_miles','2016_population_density_km','Location'])
final_table['link'] = all_links   
final_table.drop(final_table.index[[0]], inplace = True)

In [28]:
final_table.reset_index(drop = True, inplace=True)

In [29]:
final_table.head(5)

,Rank,city,state,2017_estimate,2018_census,change,2016_land_area_miles,2016_land_area_km,2016_population_density_miles,2016_population_density_km,Location,link
0,1,New York City[d],New York,"8,398,748","8,175,133",+2.74%,301.5 sq mi,780.9 km2,"28,317/sq mi","10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W﻿...,https://en.wikipedia.org/wiki/New_York_City
1,2,Los Angeles,California,"3,990,456","3,792,621",+5.22%,468.7 sq mi,"1,213.9 km2","8,484/sq mi","3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°...,https://en.wikipedia.org/wiki/Los_Angeles
2,3,Chicago,Illinois,"2,705,994","2,695,598",+0.39%,227.3 sq mi,588.7 km2,"11,900/sq mi","4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W﻿...,https://en.wikipedia.org/wiki/Chicago
3,4,Houston[3],Texas,"2,325,502","2,100,263",+10.72%,637.5 sq mi,"1,651.1 km2","3,613/sq mi","1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W﻿...,https://en.wikipedia.org/wiki/Houston
4,5,Phoenix,Arizona,"1,660,272","1,445,632",+14.85%,517.6 sq mi,"1,340.6 km2","3,120/sq mi","1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°...,"https://en.wikipedia.org/wiki/Phoenix,_Arizona"


In [30]:
links = final_table['link'].tolist()

for link in links[:]:
    response = requests.get(link)
    data = response.text
    soup = BeautifulSoup(data, 'lxml')
    city_data = soup.find('div',class_ ='infobox geography vcard')
    table = soup.findChildren('table')[0]
    rows = table.find_all('tr',class_ ='mergedrow')
    table_list_city = []
    table_list__header = []
    columns = []
    cities = []
    
    col_list_new = []
    for row in rows:
        col_list = []
        city_col = row.find_all('th')
        city_cols = [x.text.strip() for x in city_col]
       
        for i in city_cols:
            i = i.replace('•\xa0','')
            i = re.sub(r'\[.*?\]', '', i)
            col_list.append(i)
        city_col = row.find_all('td')
        city_cols_1 = [x.text.strip() for x in city_col]
        columns.append(col_list)
        cities.append(city_cols)
    
    lower_case_col = [[string.lower() for string in sublist] for sublist in columns]
    lower_case_city = [[string.lower() for string in sublist] for sublist in cities]
    
    mydict = [dict(zip(*z)) for z in zip(lower_case_col, lower_case_city)]
    result = {}
    for x in mydict:
        result.update(x)
    result_table.append(result)
    
df = pd.DataFrame(result_table)

In [32]:
df.columns

Index(['', 'a city', 'acting mayor and vice mayor', 'administrator', 'airport',
       'airports', 'alaska house', 'alaska senate', 'aldermen', 'area code',
       ...
       'urban (2010)', 'urban density', 'usda hardiness zones', 'vice mayor',
       'vice-mayor', 'virginia senate', 'water', 'waterways', 'white',
       'wikimedia commons'],
      dtype='object', length=283)

In [33]:
result_df = pd.concat([final_table, df], axis=1)

column_names = list(result_df.columns.values)
col_list = []
for i in column_names:
    col_list.append(i.replace('•\xa0',''))
result_df.columns = col_list

In [35]:
result_df.head(5)

,Rank,city,state,2017_estimate,2018_census,change,2016_land_area_miles,2016_land_area_km,2016_population_density_miles,2016_population_density_km,...,urban (2010),urban density,usda hardiness zones,vice mayor,vice-mayor,virginia senate,water,waterways,white,wikimedia commons
0,1,New York City[d],New York,"8,398,748","8,175,133",+2.74%,301.5 sq mi,780.9 km2,"28,317/sq mi","10,933/km2",...,NaN,NaN,NaN,NaN,NaN,NaN,• water,NaN,NaN,NaN
1,2,Los Angeles,California,"3,990,456","3,792,621",+5.22%,468.7 sq mi,"1,213.9 km2","8,484/sq mi","3,276/km2",...,NaN,NaN,NaN,NaN,NaN,NaN,• water,NaN,NaN,NaN
2,3,Chicago,Illinois,"2,705,994","2,695,598",+0.39%,227.3 sq mi,588.7 km2,"11,900/sq mi","4,600/km2",...,NaN,NaN,NaN,NaN,NaN,NaN,• water,NaN,NaN,NaN
3,4,Houston[3],Texas,"2,325,502","2,100,263",+10.72%,637.5 sq mi,"1,651.1 km2","3,613/sq mi","1,395/km2",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Phoenix,Arizona,"1,660,272","1,445,632",+14.85%,517.6 sq mi,"1,340.6 km2","3,120/sq mi","1,200/km2",...,NaN,NaN,NaN,NaN,NaN,NaN,• water,NaN,NaN,NaN


In [36]:
df1 = []
for link in links[:]:
    response = requests.get(link)
    data = response.text
    soup = BeautifulSoup(data, 'lxml')
    p = soup.find('div',class_ ='mw-parser-output').findAll('p')
    s = " "
    for i in range(1,3,1):
        string = s + (p[i].text)
    df1.append(s)

In [38]:
final_desc = []
for i in df1:
    i = re.sub(r'\(.*?\)', '', i)
    i = re.sub(r'\[.*?\]', '', i)
    i = re.sub(r'\/.*?\)', '', i)
    i = i.replace(')', '')
    i = i.replace('\n', '')
    i = re.sub(r'\/.*?\/', '', i)
    final_desc.append(i)

In [39]:
result_df['About_city'] = final_desc

## Data Cleaning

In [42]:
result_df.to_csv('ToposList.csv')

In [50]:
result_df['area codes'].fillna(result_df['area code(s)'])
result_df['area codes'].fillna(result_df['area code'])
result_df.loc[result_df['area codes'].isnull(),'area codes'] = result_df['area code(s)']
result_df.loc[result_df['area codes'].isnull(),'area codes'] = result_df['area code']
result_df.loc[result_df['county'].isnull(),'county'] = result_df['counties']

In [53]:
result_df = result_df.loc[:, result_df.isnull().mean() < .55]

In [55]:
result_df = result_df.loc[:, ~result_df.columns.duplicated()]
result_df.drop(columns= ['rank'], inplace=True)

In [56]:
result_df.isna().sum()


Rank                               0
city                               0
state                              0
2017_estimate                      0
2018_census                        0
change                             0
2016_land_area_miles               0
2016_land_area_km                  0
2016_population_density_miles      0
2016_population_density_km         0
Location                           0
link                               0
area codes                        38
county                            36
density                           39
gnis feature id                  139
land                              27
mayor                             72
metro                            136
summer (dst)                      34
type                              87
water                             28
About_city                         0
dtype: int64

In [57]:
result_df.drop(columns = 'gnis feature id', inplace=True)
result_df.drop(columns = 'density', inplace=True)

In [58]:
result_df.columns

Index(['Rank', 'city', 'state', '2017_estimate', '2018_census', 'change',
       '2016_land_area_miles', '2016_land_area_km',
       '2016_population_density_miles', '2016_population_density_km',
       'Location', 'link', 'area codes', 'county', 'land', 'mayor', 'metro',
       'summer (dst)', 'type', 'water', 'About_city'],
      dtype='object')

In [59]:
result_df.columns = ['rank', 'city', 'state','estimate_2017', 'census_2018', 'change', 'land_area_miles_2016', 
                     'land_area_km_2016','population_density_miles_2016', 'population_density_km_2016', 'location', 
                     'city_link', 'area_codes', 'county','area_land', 'gov_mayor', 'area_metro', 'time_zone_summer_dst',
                     'gov_type', 'area_water','about_city']

In [60]:
city_clean = []
for i in result_df['city'].tolist():
    i = i.replace('•\xa0','')
    i = re.sub(r'\[.*?\]', '', i)
    city_clean.append(i)
result_df['city'] = city_clean

In [62]:
estimate_clean = []
for i in result_df['estimate_2017'].tolist():
    i = i.replace(',','')
    estimate_clean.append(i)
result_df['estimate_2017'] = estimate_clean
result_df.rename(columns={'estimate_2017':'estimate_population_2017'}, inplace=True)

In [63]:
clean_2018_census = []
for i in result_df['census_2018'].tolist():
    i = i.replace(',','')
    clean_2018_census.append(i)
result_df['census_2018'] = clean_2018_census

In [64]:
result_df.isnull().sum()

rank                               0
city                               0
state                              0
estimate_population_2017           0
census_2018                        0
change                             0
land_area_miles_2016               0
land_area_km_2016                  0
population_density_miles_2016      0
population_density_km_2016         0
location                           0
city_link                          0
area_codes                        38
county                            36
area_land                         27
gov_mayor                         72
area_metro                       136
time_zone_summer_dst              34
gov_type                          87
area_water                        28
about_city                         0
dtype: int64

In [66]:
clean_2016_land_area_miles = []
for i in result_df['land_area_miles_2016'].tolist():
    i = i.replace('sq','')
    i = i.replace('mi','')
    i = i.replace(',','')
    clean_2016_land_area_miles.append(i)
result_df['land_area_miles_2016'] = clean_2016_land_area_miles

clean_2016_land_area_km = []
for i in result_df['land_area_km_2016'].tolist():
    i = i.replace('km2','')
    i = i.replace(',','')
    clean_2016_land_area_km.append(i)
result_df['land_area_km_2016'] = clean_2016_land_area_km

clean_2016_population_density_miles = []
for i in result_df['population_density_miles_2016'].tolist():
    i = i.replace('/sq','')
    i = i.replace('mi','')
    i = i.replace(',','')
    clean_2016_population_density_miles.append(i)
result_df['population_density_miles_2016'] = clean_2016_population_density_miles

clean_2016_population_density_km = []
for i in result_df['population_density_km_2016'].tolist():
    i = i.replace('/km2','')
    i = i.replace(',','')
    clean_2016_population_density_km.append(i)
result_df['population_density_km_2016'] = clean_2016_population_density_km

clean_area_land = []
for i in result_df['area_land'].tolist():
    i = re.sub(r'\(.*?\)', '', str(i))
    i = re.sub(r'\[.*?\]', '', str(i))
    i = i.replace('sq','')
    i = i.replace('mi','')
    i = i.replace('km2','')
    i = i.replace(',','')
    clean_area_land.append(i)
result_df['area_land'] = clean_area_land

clean_area_metro = []
for i in result_df['area_metro'].tolist():
    i = re.sub(r'\(.*?\)', '', str(i))
    i = re.sub(r'\[.*?\]', '', str(i))
    i = i.replace('sq','')
    i = i.replace('mi','')
    i = i.replace(',','')
    clean_area_metro.append(i)
result_df['area_metro'] = clean_area_metro

clean_area_water = []
for i in result_df['area_water'].tolist():
    i = re.sub(r'\(.*?\)', '', str(i))
    i = i.replace('sq','')
    i = i.replace('mi','')
    i = i.replace('km2','')
    i = i.replace(',','')
    clean_area_water.append(i)
result_df['area_water'] = clean_area_water

result_df['area_water'] = result_df['area_water'].str.split(' ').str[0]

In [68]:
clean_change = []
for i in result_df['change'].tolist():
    i = i.replace('%','')
    clean_change.append(i)
result_df['change'] = clean_change

clean_location = []
for i in result_df['location'].tolist():
    i = re.sub(r'\(.*?\)', '', str(i))
    clean_location.append(i)
result_df['location'] = clean_location

clean_gov_mayor = []
for i in result_df['gov_mayor'].tolist():
    i = re.sub(r'\(.*?\)', '', str(i))
    i = re.sub(r'\[.*?\]', '', str(i))
    clean_gov_mayor.append(i)
result_df['gov_mayor'] = clean_gov_mayor

clean_gov_type = []
for i in result_df['gov_type'].tolist():
    i = re.sub(r'\(.*?\)', '', str(i))
    i = re.sub(r'\[.*?\]', '', str(i))
    clean_gov_type.append(i)
result_df['gov_type'] = clean_gov_type

clean_area_codes = []
result_df['area_codes'] = result_df['area_codes'].astype(str)
for i in result_df['area_codes'].tolist():
    if i != '':
        #print(i)
        i = i.replace(', and','/')
        i = i.replace(',','/')
        i = i.replace(' and ','/')
        i = i.replace(' & ','/')
        i = i.replace('both ','')
        i = i.replace('east: ','')
        i = i.replace('central: ','/')
        i = i.replace('west: ','/')
        i = re.sub(r'\(.*?\)', '', str(i))
        i = re.sub(r'\[.*?\]', '', str(i))
        clean_area_codes.append(i)
    else:
        clean_area_codes.append('') 
#print(len(clean_area_codes))
result_df['area_codes'] = clean_area_codes

lattitude = []
longitude = []
result_df['location'] = result_df['location'].astype(str)
for i in result_df['location'].tolist():
    lattitude.append(i[-20:-13])
    longitude.append(i[-11:-2])
result_df['lattitude'] = lattitude
result_df['longitude'] = longitude
result_df.drop(columns = 'location', inplace=True)


In [69]:
clean_county = []
for i in result_df['county'].tolist():
    i = re.sub(r'\[.*?\]', '', str(i))
    clean_county.append(i)
result_df['county'] = clean_county

result_df['country'] = 'USA'

In [73]:
result_df.drop(columns= 'time_zone_summer_dst', inplace = True)

In [74]:
result_df.isnull().sum()

rank                             0
city                             0
state                            0
estimate_population_2017         0
census_2018                      0
change                           0
land_area_miles_2016             0
land_area_km_2016                0
population_density_miles_2016    0
population_density_km_2016       0
city_link                        0
area_codes                       0
county                           0
area_land                        0
gov_mayor                        0
area_metro                       0
gov_type                         0
area_water                       0
about_city                       0
lattitude                        0
longitude                        0
country                          0
dtype: int64

In [76]:
result_df.set_index('rank', inplace=True)
result_df.head(5)

,city,state,estimate_population_2017,census_2018,change,land_area_miles_2016,land_area_km_2016,population_density_miles_2016,population_density_km_2016,city_link,...,county,area_land,gov_mayor,area_metro,gov_type,area_water,about_city,lattitude,longitude,country
rank,,,,,,,,,,,,,,,,,,,,,
1,New York City,New York,8398748,8175133,+2.74,301.5,780.9,28317,10933,https://en.wikipedia.org/wiki/New_York_City,...,nan,• land,• mayor,• metro,• type,• water,,40.663,-73.9387,USA
2,Los Angeles,California,3990456,3792621,+5.22,468.7,1213.9,8484,3276,https://en.wikipedia.org/wiki/Los_Angeles,...,county,• land,• mayor,• metro,• type,• water,,34.0194,-118.4108,USA
3,Chicago,Illinois,2705994,2695598,+0.39,227.3,588.7,11900,4600,https://en.wikipedia.org/wiki/Chicago,...,counties,• land,• mayor,• metro,• type,• water,,41.837,-87.6818,USA
4,Houston,Texas,2325502,2100263,+10.72,637.5,1651.1,3613,1395,https://en.wikipedia.org/wiki/Houston,...,counties,• land,• mayor,• metro,• type,nan,,29.786,-95.3909,USA
5,Phoenix,Arizona,1660272,1445632,+14.85,517.6,1340.6,3120,1200,"https://en.wikipedia.org/wiki/Phoenix,_Arizona",...,county,• land,• mayor,• metro,• type,• water,,33.5722,-112.0901,USA
